In [15]:
import os
import openai
import json
openai_api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = openai_api_key

In [71]:
!pip show psycopg2

Name: psycopg2
Version: 2.9.10
Summary: psycopg2 - Python-PostgreSQL Database Adapter
Home-page: https://psycopg.org/
Author: Federico Di Gregorio
Author-email: fog@initd.org
License: LGPL with exceptions
Location: E:\llm_fin_2\venv\Lib\site-packages
Requires: 
Required-by: 


In [93]:
class Text2SQL:
    def __init__(self, model="gpt-4o-mini", engine=None, db_structure=None, acc_name=None):
        self.llm = ChatOpenAI(model=model)
        self.sql_engine = engine
        self.sql_database = SQLDatabase(engine=engine)  # Initialize database with engine
        self.db_structure = db_structure  # Database structure as a JSON-like dict
        self.acc_name = acc_name  # Account names list or description

    def chain_of_thought_prompt(self, question):
        prompt_of_cot = f"""
        Bạn là một chuyên gia phân tích tài chính chuyên sâu. Hãy phân tích câu hỏi dưới đây và thực hiện các bước để trả lời chính xác.

        Câu hỏi: "{question}"

        Bước 1: Xác định loại vấn đề cần phân tích và chia nhỏ vấn đề
        - Phân tích yêu cầu và chia nhỏ bài toán: [Xác định các vấn đề cần phân tích từ câu hỏi]
        - Mục cụ thể trong báo cáo: [Từ những vấn đề đã chia nhỏ ở trên, xác định mục cụ thể cần tìm trong báo cáo (ví dụ: doanh thu, lợi nhuận ròng, tổng tài sản). Nếu là các chỉ số tài chính, xác định thành phần cụ thể để tính toán]
        - Loại báo cáo: [Xác định loại báo cáo tài chính liên quan (Bảng cân đối kế toán, Báo cáo kết quả kinh doanh, Báo cáo lưu chuyển tiền tệ)]
        
        Ví dụ ở bước 1 về tính thanh khoản của BID năm 2022:  
        - Thành phần cần tính toán:
        - Current Ratio: `Tổng Tài Sản Ngắn Hạn / Nợ Ngắn Hạn`
        - Quick Ratio: `(Tài Sản Ngắn Hạn - Hàng Tồn Kho) / Nợ Ngắn Hạn`
        - Cash Ratio: `(Tiền Mặt + Tài Sản Tương Đương Tiền) / Nợ Ngắn Hạn`
        Tất cả các mục được xác nhận trong ví dụ ở trong Balance_sheet

        Bước 2: Xác định thông tin cơ bản từ vấn đề ở bước 1
        - Tên công ty: [Xác định những tên công ty được đề cập]
        - Năm tài chính: [Xác định những năm tài chính được nhắc đến]
        - Kỳ báo cáo: [Xác định kỳ báo cáo cụ thể nếu có, nếu không mặc định là "Cả năm" (Ví dụ: 1, 2, "Cả năm", 'Nửa năm')]

        Bước 3: Xác định thông tin từng vấn đề nhỏ
        - Với lần lượt từng vấn đề, hãy viết nó lại thành 1 câu query với cấu trúc như sau: 
        'Tìm những [tài khoản đã xác nhận] ở [table đã xác nhận] của [ngân hàng nào (nếu có)] để trả về kết quả [vấn đề nhỏ]'

        Bước 4: Trả về câu truy vấn SQL dựa trên các thông tin đã xác định ở bước 1, 2 và 3
        - Đây là cấu trúc database của bạn cần sử dụng: {json.dumps(self.db_structure)}
        - Đây là danh sách tên các tài khoản bạn cần {self.acc_name}. Hãy dựa trên đấy để tạo ra câu truy vấn SQL

        Bắt đầu với Bước 1 và làm theo từng bước trên để cung cấp câu trả lời chi tiết.
        """
        return prompt_of_cot

    def get_sql_query(self, input_question):
        # Generate chain of thought prompt
        prompt_of_cot = self.chain_of_thought_prompt(input_question)
        
        # Chain to generate SQL query using LLM
        write_query = create_sql_query_chain(self.llm, self.sql_database)
        
        # Chain to generate and return only the SQL query
        chain = RunnablePassthrough(assign={'question': prompt_of_cot, 'query': write_query})

        # Execute query and get the result
        result = chain.invoke({"question": prompt_of_cot})  # Use the prompt itself
        
        # Debugging: Print the result to see its structure
        print("Result from chain.invoke:", result)
        
        # Check if 'query' exists in the result, else handle the error
        if 'query' in result:
            return result['query']
        else:
            raise KeyError(f"The key 'query' was not found in the result. Available keys: {list(result.keys())}")

# Usage example
message = "Chọn ra ngân hàng có ROE cao nhất năm 2024 quý 2"
text2sql_instance = Text2SQL(model="gpt-4o-mini", engine=sql_engine, db_structure=db_structure, acc_name=acc_name)

try:
    sql_query = text2sql_instance.get_sql_query(message)
    print("Generated SQL Query:")
    print(sql_query)
except KeyError as e:
    print("Error:", e)


e:\llm_fin_2\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'vector' of column 'embedding'
  self._metadata.reflect(
e:\llm_fin_2\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'vector' of column 'bankembedding'
  self._metadata.reflect(


Result from chain.invoke: {'question': '\n        Bạn là một chuyên gia phân tích tài chính chuyên sâu. Hãy phân tích câu hỏi dưới đây và thực hiện các bước để trả lời chính xác.\n\n        Câu hỏi: "Chọn ra ngân hàng có ROE cao nhất năm 2024 quý 2"\n\n        Bước 1: Xác định loại vấn đề cần phân tích và chia nhỏ vấn đề\n        - Phân tích yêu cầu và chia nhỏ bài toán: [Xác định các vấn đề cần phân tích từ câu hỏi]\n        - Mục cụ thể trong báo cáo: [Từ những vấn đề đã chia nhỏ ở trên, xác định mục cụ thể cần tìm trong báo cáo (ví dụ: doanh thu, lợi nhuận ròng, tổng tài sản). Nếu là các chỉ số tài chính, xác định thành phần cụ thể để tính toán]\n        - Loại báo cáo: [Xác định loại báo cáo tài chính liên quan (Bảng cân đối kế toán, Báo cáo kết quả kinh doanh, Báo cáo lưu chuyển tiền tệ)]\n        \n        Ví dụ ở bước 1 về tính thanh khoản của BID năm 2022:  \n        - Thành phần cần tính toán:\n        - Current Ratio: `Tổng Tài Sản Ngắn Hạn / Nợ Ngắn Hạn`\n        - Quick Rat

In [94]:
    def get_sql_query(input_question):
        # Generate chain of thought prompt
        prompt_of_cot = chain_of_thought_prompt(input_question)
        
        # Chain to generate SQL query using LLM
        write_query = create_sql_query_chain(
            llm, sql_database
        )
        return write_query
message_cot = get_sql_query(message)
message_cot

RunnableAssign(mapper={
  input: RunnableLambda(...),
  table_info: RunnableLambda(...)
})
| RunnableLambda(lambda x: {k: v for k, v in x.items() if k not in ('question', 'table_names_to_use')})
| PromptTemplate(input_variables=['input', 'table_info'], input_types={}, partial_variables={'top_k': '5'}, template='You are a PostgreSQL expert. Given an input question, first create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\nPay attention to use only the column names you ca

In [18]:
import psycopg2
# import numpy as np
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
user = os.getenv('USER')
host = os.getenv('HOST')
port = os.getenv('PORT')
password = os.getenv('PASSWORD')
dbname = os.getenv('DB1')

# Set up PostgreSQL connection
connection = psycopg2.connect(
    host=host,
    port=port,
    dbname=dbname,
    user=user,
    password=password
)
cursor = connection.cursor()

In [20]:
!pip install sqlalchemy


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import URL

sqlalchemy_connection_url = URL.create(
    "postgresql+psycopg2",
    username=user,
    password=password,
    host=host,
    database=dbname,
)
sql_engine = create_engine(sqlalchemy_connection_url)

In [22]:
# Tạo một dictionary để lưu thông tin cấu trúc database
db_structure = {}

inspector = inspect(sql_engine)
tables = inspector.get_table_names()
db_structure['tables'] = tables

# Lưu thông tin chi tiết của từng bảng
for table in tables:
    columns = inspector.get_columns(table)
    db_structure[table] = {
        'columns': [{'name': column['name'], 'type': str(column['type'])} for column in columns]
    }

# In ra để kiểm tra (tùy chọn)
print(json.dumps(db_structure))

C:\Users\nn200\AppData\Local\Temp\ipykernel_29544\3848115435.py:10: SAWarning: Did not recognize type 'vector' of column 'bankembedding'
  columns = inspector.get_columns(table)
C:\Users\nn200\AppData\Local\Temp\ipykernel_29544\3848115435.py:10: SAWarning: Did not recognize type 'vector' of column 'embedding'
  columns = inspector.get_columns(table)


{"tables": ["BANK", "BALANCESHEET", "INCOMESTATEMENT", "CHUNK", "explanation", "CASHFLOW", "METADATA", "ACCNO"], "BANK": {"columns": [{"name": "bankid", "type": "INTEGER"}, {"name": "bankname", "type": "TEXT"}, {"name": "symbol", "type": "TEXT"}, {"name": "abbreviation", "type": "TEXT"}, {"name": "banknameeng", "type": "VARCHAR(255)"}, {"name": "bankembedding", "type": "NULL"}]}, "BALANCESHEET": {"columns": [{"name": "id", "type": "TEXT"}, {"name": "reportid", "type": "TEXT"}, {"name": "accountno", "type": "INTEGER"}, {"name": "accountname", "type": "TEXT"}, {"name": "amount", "type": "NUMERIC(15, 2)"}]}, "INCOMESTATEMENT": {"columns": [{"name": "id", "type": "TEXT"}, {"name": "reportid", "type": "TEXT"}, {"name": "accountno", "type": "INTEGER"}, {"name": "accountname", "type": "TEXT"}, {"name": "amount", "type": "NUMERIC(15, 2)"}]}, "CHUNK": {"columns": [{"name": "id", "type": "UUID"}, {"name": "reportid", "type": "TEXT"}, {"name": "text", "type": "TEXT"}, {"name": "embedding", "type"

In [13]:
!pip install langchain_community

  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached SQLAlchemy-2.0.35-cp312-cp312-win_amd64.whl.metadata (9.9 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
     ---------------------------------------- 0.0/52.0 kB ? eta -:--:--
     ------- -------------------------------- 10.2/52.0 kB ? eta -:--:--
     ---------------------- --------------- 30.7/52.0 kB 262.6 kB/s eta 0:00:01
     ---------------------- --------------- 30.7/52.0 kB 262.6 kB/s eta 0:00:01
     ---------------------- --------------- 30.7/52.0 kB 262.6 kB/s eta 0:00:01
     -------------------------------------  51.2/52.0 

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement langchain_community.chains (from versions: none)
ERROR: No matching distribution found for langchain_community.chains

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
!pip install langchain_openai


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached tiktoken-0.8.0-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached regex-2024.9.11-cp312-cp312-win_amd64.whl.metadata (41 kB)
   ---------------------------------------- 0.0/50.4 kB ? eta -:--:--
   -------- ------------------------------- 10.2/50.4 kB ? eta -:--:--
   ---------------------------------------- 50.4/50.4 kB 514.5 kB/s eta 0:00:00
   ---------------------------------------- 0.0/389.3 kB ? eta -:--:--
   ------ --------------------------------- 61.4/389.3 kB 1.7 MB/s eta 0:00:01
   ------------------- -------------------- 194.6/389.3 kB 2.0 MB/s eta 0:00:01
   ---------------------------- ----------- 276.5/389.3 kB 1.9 MB/s eta 0:00:01
   ---------------------------------------  389.1/389.3 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 389.3/389.3 kB 1.9 MB/s eta 0:00:00
Using cached tiktoken-0.8.0-cp312-cp312-win_amd64.whl (883 kB)
Using cached distro-1.9.0-py3-none

In [23]:
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_openai import ChatOpenAI

In [24]:
!pip install voyageai


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
import voyageai

In [69]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
# Template for answering the question based on SQL execution
system_role = """
Given the following user question, corresponding SQL query, and SQL result, answer the user question.
Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer:
"""
# Tạo đối tượng SQLDatabase từ sql_engine
sql_database = SQLDatabase(engine=sql_engine)
# Tool for executing SQL queries on the db
execute_query = QuerySQLDataBaseTool(db=sql_database)

# Chain to write an SQL query using LLM
write_query = create_sql_query_chain(
    llm, sql_database  # llm is the language model, db is the SQL database connection
)

# Defining the answer prompt using the system role template
answer_prompt = PromptTemplate.from_template(system_role)

# Parser for converting the LLM output to a string
answer = answer_prompt | llm | StrOutputParser()

e:\llm_fin_2\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'vector' of column 'embedding'
  self._metadata.reflect(
e:\llm_fin_2\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'vector' of column 'bankembedding'
  self._metadata.reflect(


In [29]:
sql_query = """
SELECT accname
FROM "ACCNO"
"""
acc_name = sql_database.run(sql_query)
# Thực thi truy vấn và tạo câu trả lời
try:
    sql_result = sql_database.run(sql_query)
    answer_input = {
        "question": 'Lấy tên các tài khoản',
        "query": sql_query,
        "result": sql_result
    }
    final_answer = answer.invoke(answer_input)

    print("SQL Result:")
    print(sql_result)
except Exception as e:
    print(f"Error: {str(e)}")

SQL Result:
[('Cash Flows from Operating Activities',), ('Interest income and similar earnings received',), ('Interest expenses and similar expenses paid',), ('Net income from services received',), ('Foreign exchange, gold, and securities trading gains/losses',), ('Other expenses paid',), ('Cash for other handled and risk-provisioned accounts',), ('Cash paid to employees in management activities',), ('Corporate income tax paid in the year',), ('Net cash flow from operating activities before changes in assets and liabilities',), ('Increase/Decrease in Operating Assets',), ('Deposits to other credit institutions',), ('Investment in securities',), ('Loans to customers and finance leases',), ('Customer deposits',), ('Provision for loan losses',), ('Other operating assetsIncrease/Decrease in Liabilities',), ('Deposits from the State Bank of Vietnam',), ('Deposits from other credit institutions',), ('Deposits from customers',), ('Grants and investments received',), ('Payables and accrued exp

In [ ]:
def chain_of_thought_prompt(question):
    prompt_of_cot = f"""
    Bạn là một chuyên gia phân tích tài chính chuyên sâu. Hãy phân tích câu hỏi dưới đây và thực hiện các bước để trả lời chính xác.

    Câu hỏi: "{question}"

    Bước 1: Xác định loại vấn đề cần phân tích và chia nhỏ vấn đề
    - Phân tích yêu cầu và chia nhỏ bài toán: [Xác định các vấn đề cần phân tích từ câu hỏi]
    - Mục cụ thể trong báo cáo: [Từ những vấn để đã chia nhỏ ở trên, xác định mục cụ thể cần tìm trong báo cáo (ví dụ: doanh thu, lợi nhuận ròng, tổng tài sản). Nếu là các chỉ số tài chính, xác định thành phần cụ thể để tính toán]
    - Loại báo cáo: [Xác định loại báo cáo tài chính liên quan (Bảng cân đối kế toán, Báo cáo kết quả kinh doanh, Báo cáo lưu chuyển tiền tệ)]
    Ví dụ ở bước 1 về về tính thanh khoản của BID năm 2022:  
    - Thành phần cần tính toán:
      - Current Ratio: `Tổng Tài Sản Ngắn Hạn / Nợ Ngắn Hạn`
      - Quick Ratio: `(Tài Sản Ngắn Hạn - Hàng Tồn Kho) / Nợ Ngắn Hạn`
      - Cash Ratio: `(Tiền Mặt + Tài Sản Tương Đương Tiền) / Nợ Ngắn Hạn`
      Tất cả các mục được xác nhận trong ví dụ ở trong Balance_sheet
   
     Bước 2: Xác định thông tin cơ bản từ vấn đề ở bước 1
    - Tên công ty: [Xác định những tên công ty được đề cập]
    - Năm tài chính: [Xác định những năm tài chính được nhắc đến]
    - Kỳ báo cáo: [Xác định kỳ báo cáo cụ thể nếu có, nếu không mặc định là "Cả năm" (Ví dụ: 1, 2, "Cả năm", 'Nửa năm')]

    Bước 3: Xác định thông tin từng vấn đề nhỏ
    - Với lần lượt từng vấn đề, hãy viết nó lại thành 1 câu query với cấu trúc như sau: 
    'Tìm những [tài khoản đã xác nhận] ở [table đã xác nhận] của [ngân hàng nào (nêú có)] để trả về kết quả [vấn đề nhỏ]'
    Bước 4: Trả về câu truy vấn SQL dựa trên các thông tin đã xác định ở bước 1, 2 và 3
    - Đây là cấu trúc database của bạn cần sử dụng: {json.dumps(db_structure)}
    - Đây là danh sách tên các tài khoản bạn cần {acc_name}. Hãy dựa trên đấy để tạo ra câu truy vấn SQL

    Bắt đầu với Bước 1 và làm theo từng bước trên để cung cấp câu trả lời chi tiết.
    """
    return prompt_of_cot

# Sử dụng hàm
message = "Chọn ra ngân hàng có ROE cao nhất năm 2024 quý 2"
message_cot = chain_of_thought_prompt(message)
print(message_cot)


    Bạn là một chuyên gia phân tích tài chính chuyên sâu. Hãy phân tích câu hỏi dưới đây và thực hiện các bước để trả lời chính xác.

    Câu hỏi: "Chọn ra ngân hàng có ROE cao nhất năm 2024 quý 2"

    Bước 1: Xác định loại vấn đề cần phân tích và chia nhỏ vấn đề
    - Phân tích yêu cầu và chia nhỏ bài toán: [Xác định các vấn đề cần phân tích từ câu hỏi]
    - Mục cụ thể trong báo cáo: [Từ những vấn để đã chia nhỏ ở trên, xác định mục cụ thể cần tìm trong báo cáo (ví dụ: doanh thu, lợi nhuận ròng, tổng tài sản). Nếu là các chỉ số tài chính, xác định thành phần cụ thể để tính toán]
    - Loại báo cáo: [Xác định loại báo cáo tài chính liên quan (Bảng cân đối kế toán, Báo cáo kết quả kinh doanh, Báo cáo lưu chuyển tiền tệ)]
    Ví dụ ở bước 1 về về tính thanh khoản của BID năm 2022:  
    - Thành phần cần tính toán:
      - Current Ratio: `Tổng Tài Sản Ngắn Hạn / Nợ Ngắn Hạn`
      - Quick Ratio: `(Tài Sản Ngắn Hạn - Hàng Tồn Kho) / Nợ Ngắn Hạn`
      - Cash Ratio: `(Tiền Mặt + Tài Sản 

In [65]:
# Chain to generate and return only the SQL query
chain = RunnablePassthrough.assign(query=write_query)

# Hàm để trả về truy vấn SQL
def get_sql_query(input_question):
    result = chain.invoke({"question": input_question})
    return result['query']

sql_query = get_sql_query(message_cot)

print("Generated SQL Query:")
print(sql_query)

Generated SQL Query:
Question: 
    Bạn là một chuyên gia phân tích tài chính chuyên sâu. Hãy phân tích câu hỏi dưới đây và thực hiện các bước để trả lời chính xác.

    Câu hỏi: "Chọn ra ngân hàng có ROE cao nhất năm 2024 quý 2"

Bước 1: Xác định loại vấn đề cần phân tích và chia nhỏ vấn đề
- Phân tích yêu cầu và chia nhỏ bài toán: Cần xác định ROE (Return on Equity) của các ngân hàng trong năm 2024, quý 2.
- Mục cụ thể trong báo cáo: Để tính ROE, cần tìm:
  - Lợi nhuận ròng (Net profit): Có thể lấy từ báo cáo kết quả kinh doanh (INCOMESTATEMENT).
  - Vốn chủ sở hữu (Total equity): Có thể lấy từ bảng cân đối kế toán (BALANCESHEET).
- Loại báo cáo: Cần xem xét cả hai loại báo cáo: Bảng cân đối kế toán và Báo cáo kết quả kinh doanh.

Bước 2: Xác định thông tin cơ bản từ vấn đề ở bước 1
- Tên ngân hàng: Sẽ xác định từ bảng BANK.
- Năm tài chính: 2024.
- Kỳ báo cáo: Quý 2.

Bước 3: Xác định thông tin từng vấn đề nhỏ
- Tìm lợi nhuận ròng ở INCOMESTATEMENT của ngân hàng nào để trả về kết qu

In [33]:
import voyageai
voyage_api = os.getenv('VOYAGE_API')
if not voyage_api:
    raise EnvironmentError("VOYAGE_API environment variable not set.")


In [34]:
# Initialize VoyageAI client
vo = voyageai.Client(api_key=voyage_api)
# Embed the query text
query_embedding = vo.embed("vàng", model="voyage-finance-2")  # Assuming embed returns a list of embeddings

In [35]:
from database.vectorDB import VectorDB
from dotenv import load_dotenv
import os

load_dotenv()

user = os.getenv('USER')
host = os.getenv('HOST')
port = os.getenv('PORT')
password = os.getenv('PASSWORD')
dbname = os.getenv('DB1')

db_config = {
    'user': user,
    'password': password,
    'host': host,
    'port': port,
    'dbname': dbname
}


vectordb = VectorDB(**db_config)

In [36]:
query_text =[]

In [6]:
vectordb.query_with_distance(
    query_text, 
    table_name="ACCNO", 
    distance='cosine',
    columns = ['accno','accname']
)

[[{'accno': 386, 'accname': 'Goodwill', 'distance': 0.31812524795532227},
  {'accno': 7, 'accname': 'Income', 'distance': 0.32128214836120605},
  {'accno': 10,
   'accname': 'Cash, silver, gold and gemstones',
   'distance': 0.35844069719314575},
  {'accno': 38, 'accname': 'Other assets ', 'distance': 0.37384683255971707},
  {'accno': 30, 'accname': 'Fixed assets', 'distance': 0.3776112609389798}],
 [{'accno': 4,
   'accname': 'Total liabilities',
   'distance': 0.020470262793912508},
  {'accno': 5,
   'accname': 'Total liabilities and equity',
   'distance': 0.07502073496703554},
  {'accno': 1, 'accname': 'Total assets', 'distance': 0.11852972993077304},
  {'accno': 48,
   'accname': 'Other liabilities',
   'distance': 0.14183884013659565},
  {'accno': 6, 'accname': 'Total equity', 'distance': 0.18326897521429963}]]

In [90]:
def chain_of_thought_prompt(question):
    prompt_of_cot = f"""
    Bạn là một chuyên gia phân tích tài chính chuyên sâu. Hãy phân tích câu hỏi dưới đây và thực hiện các bước để trả lời chính xác.
    Câu hỏi: "{question}"
    Bước 1: Xác định loại vấn đề cần phân tích và chia nhỏ vấn đề
    - Phân tích yêu cầu và chia nhỏ bài toán: [Xác định các vấn đề cần phân tích từ câu hỏi]
    - Mục cụ thể trong báo cáo: [Từ những vấn để đã chia nhỏ ở trên, xác định mục cụ thể cần tìm trong báo cáo (ví dụ: doanh thu, lợi nhuận ròng, tổng tài sản). Nếu là các chỉ số tài chính, xác định thành phần cụ thể để tính toán]
    - Loại báo cáo: [Xác định loại báo cáo tài chính liên quan (Bảng cân đối kế toán, Báo cáo kết quả kinh doanh, Báo cáo lưu chuyển tiền tệ)]
    Ví dụ ở bước 1 về về tính thanh khoản của BID năm 2022:  
    - Thành phần cần tính toán:
      - Current Ratio: `Tổng Tài Sản Ngắn Hạn / Nợ Ngắn Hạn`
      - Quick Ratio: `(Tài Sản Ngắn Hạn - Hàng Tồn Kho) / Nợ Ngắn Hạn`
      - Cash Ratio: `(Tiền Mặt + Tài Sản Tương Đương Tiền) / Nợ Ngắn Hạn`
      Ví dụ tất cả các mục được xác nhận ở trong Balance_sheet
     Bước 2: Xác định thông tin cơ bản từ vấn đề ở bước 1
    - Tên công ty: [Xác định những tên công ty được đề cập]
    - Năm tài chính: [Xác định những năm tài chính được nhắc đến]
    - Kỳ báo cáo: [Xác định kỳ báo cáo cụ thể nếu có, nếu không mặc định là "Cả năm" (Ví dụ: 1, 2, "Cả năm", 'Nửa năm')]
    Bước 3: Nối những tên cần nhập giống trong database
    - Đảm bảo những tài khoản cần xác nhận là đã được chia nhỏ ra 
    - Dịch những tài khoản đó sang tiếng anh
    - Hãy trả về dictionary có định dạng như sau:
    'bank':'ngân hàng cần chọn', 'year':'năm cần chọn','main_table':'bảng cần chọn','account':'tài khoản cần chọn'
    """
    return prompt_of_cot

# Sử dụng hàm
message = "Chọn ra ngân hàng có ROE cao nhất năm 2024 quý 2"
message_cot = chain_of_thought_prompt(message)
print(message_cot)


    Bạn là một chuyên gia phân tích tài chính chuyên sâu. Hãy phân tích câu hỏi dưới đây và thực hiện các bước để trả lời chính xác.
    Câu hỏi: "Chọn ra ngân hàng có ROE cao nhất năm 2024 quý 2"
    Bước 1: Xác định loại vấn đề cần phân tích và chia nhỏ vấn đề
    - Phân tích yêu cầu và chia nhỏ bài toán: [Xác định các vấn đề cần phân tích từ câu hỏi]
    - Mục cụ thể trong báo cáo: [Từ những vấn để đã chia nhỏ ở trên, xác định mục cụ thể cần tìm trong báo cáo (ví dụ: doanh thu, lợi nhuận ròng, tổng tài sản). Nếu là các chỉ số tài chính, xác định thành phần cụ thể để tính toán]
    - Loại báo cáo: [Xác định loại báo cáo tài chính liên quan (Bảng cân đối kế toán, Báo cáo kết quả kinh doanh, Báo cáo lưu chuyển tiền tệ)]
    Ví dụ ở bước 1 về về tính thanh khoản của BID năm 2022:  
    - Thành phần cần tính toán:
      - Current Ratio: `Tổng Tài Sản Ngắn Hạn / Nợ Ngắn Hạn`
      - Quick Ratio: `(Tài Sản Ngắn Hạn - Hàng Tồn Kho) / Nợ Ngắn Hạn`
      - Cash Ratio: `(Tiền Mặt + Tài Sản Tư

In [ ]:
normal_chain = (
    RunnablePassthrough.assign(result=lambda x: execute_query(x["query"]))
    | answer
)

# Hàm để lấy câu trả lời bình thường từ LLM
def get_normal_answer(input_question):
    try:
        return normal_chain.invoke({"query": input_question})
    except Exception as e:
        return f"Có lỗi xảy ra: {str(e)}"

# Sử dụng hàm để lấy câu trả lời
response = get_normal_answer(message_cot)
print("Câu trả lời:")
print(response)


C:\Users\nn200\AppData\Local\Temp\ipykernel_29544\3054927349.py:3: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  RunnablePassthrough.assign(result=lambda x: execute_query(x["query"]))  # Thực thi query


Câu trả lời:
Có lỗi xảy ra: "Input to PromptTemplate is missing variables {'question'}.  Expected: ['query', 'question', 'result'] Received: ['query', 'result']\nNote: if you intended {question} to be part of the string and not a variable, please escape it with double curly braces like: '{{question}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT"


In [ ]:
chain = (
    RunnablePassthrough.assign(query=write_query).assign(result=itemgetter("query") | execute_query)  # Thực thi query
    | answer  # Chuyển kết quả thành câu trả lời
)

# Hàm để lấy câu trả lời từ LLM
def get_answer(input_question):
    try:
        result = chain.invoke({"question": input_question})
        return result
    except Exception as e:
        return f"Có lỗi xảy ra: {str(e)}"

# Sử dụng hàm để lấy câu trả lời
response = get_answer(message_cot)
print("Câu trả lời:")
print(response)

In [ ]:
    Bước 3: Trả về câu truy vấn SQL dựa trên các thông tin đã xác định ở bước 1 và 2
    - Đây là cấu trúc database của bạn cần sử dụng: {json.dumps(db_structure)}
    - Đây là danh sách tên các tài khoản bạn cần {acc_name}. Hãy dựa trên đấy để tạo ra câu truy vấn SQL

    Bắt đầu với Bước 1 và làm theo từng bước trên để cung cấp câu trả lời chi tiết.

In [ ]:
# Query with cosing distance
results = db.query_with_distance(
    query_text, 
    embedder, 
    table_name="chunkEmbedding", 
    distance='cosine',
    columns=["id", "reportId", "text"], 
    limit=5
)

print(results)

In [103]:
!pip install voyageai

In [66]:
# Định nghĩa SQL query
sql_query = """
WITH income AS (
    SELECT "reportid", "amount" AS profit 
    FROM "INCOMESTATEMENT" 
    WHERE "accountname" = 'Profit after tax' 
      AND "reportid" IN (SELECT "reportid" 
                         FROM "METADATA" 
                         WHERE "year" = 2024 AND "quarter" = 2)
),
equity AS (
    SELECT "reportid", "amount" AS equity 
    FROM "BALANCESHEET" 
    WHERE "accountname" = 'Total equity' 
      AND "reportid" IN (SELECT "reportid" 
                         FROM "METADATA" 
                         WHERE "year" = 2024 AND "quarter" = 2)
)
SELECT m."banksymbol", 
       (i.profit / e.equity) * 100 AS ROE 
FROM income i 
JOIN equity e ON i."reportid" = e."reportid" 
JOIN "METADATA" m ON i."reportid" = m."reportid" 
ORDER BY ROE DESC 
LIMIT 1;
"""

# Thực thi truy vấn và tạo câu trả lời
try:
    sql_result = sql_database.run(sql_query)
    answer_input = {
        "question": "Lấy ngân hàng có ROE cao nhất năm 2024 quý 2",
        "query": sql_query,
        "result": sql_result
    }
    final_answer = answer.invoke(answer_input)

    print("SQL Query:")
    print(sql_query)
    print("\nSQL Result:")
    print(sql_result)
    print("\nFinal Answer:")
    print(final_answer)
except Exception as e:
    print(f"Error: {str(e)}")

SQL Query:

WITH income AS (
    SELECT "reportid", "amount" AS profit 
    FROM "INCOMESTATEMENT" 
    WHERE "accountname" = 'Profit after tax' 
      AND "reportid" IN (SELECT "reportid" 
                         FROM "METADATA" 
                         WHERE "year" = 2024 AND "quarter" = 2)
),
equity AS (
    SELECT "reportid", "amount" AS equity 
    FROM "BALANCESHEET" 
    WHERE "accountname" = 'Total equity' 
      AND "reportid" IN (SELECT "reportid" 
                         FROM "METADATA" 
                         WHERE "year" = 2024 AND "quarter" = 2)
)
SELECT m."banksymbol", 
       (i.profit / e.equity) * 100 AS ROE 
FROM income i 
JOIN equity e ON i."reportid" = e."reportid" 
JOIN "METADATA" m ON i."reportid" = m."reportid" 
ORDER BY ROE DESC 
LIMIT 1;


SQL Result:
[('TCB', Decimal('17.67024387006810235000'))]

Final Answer:
Ngân hàng có ROE cao nhất năm 2024 quý 2 là TCB với ROE là 17.67%.


In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_short_term_assets",
            "description": "Retrieve all short-term asset accounts with account numbers from 10 to 39 from the ACCNO table",
            "parameters": {
                "type": "object",
                "properties": {
                    "account_range": {
                        "type": "object",
                        "properties": {
                            "start": {
                                "type": "integer",
                                "description": "Starting account number (10)",
                            },
                            "end": {
                                "type": "integer",
                                "description": "Ending account number (39)",
                            }
                        },
                        "description": "Range of account numbers to search for short-term assets"
                    },
                    "include_details": {
                        "type": "boolean",
                        "description": "Whether to include additional account details in the response",
                    }
                },
                "required": ["accno"]
            }
        }
    }
]

In [ ]:
chain = (
    RunnablePassthrough.assign(query=write_query)  # Tạo SQL query
    .assign(result=itemgetter("query") | execute_query)  # Thực thi query
    | answer  # Chuyển kết quả thành câu trả lời
)

# Hàm để lấy câu trả lời từ LLM
def get_answer(input_question):
    try:
        result = chain.invoke({"question": input_question})
        return result
    except Exception as e:
        return f"Có lỗi xảy ra: {str(e)}"

# Sử dụng hàm để lấy câu trả lời
response = get_answer(message_cot)
print("Câu trả lời:")
print(response)

In [88]:
import openai
import json
from typing import Dict, Any

# 1. Định nghĩa tools như đã có ở trên
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_short_term_assets",
            "description": "Retrieve all short-term asset accounts with account numbers from 10 to 39 from the ACCNO table",
            "parameters": {
                "type": "object",
                "properties": {
                    "account_range": {
                        "type": "object",
                        "properties": {
                            "start": {
                                "type": "integer",
                                "description": "Starting account number (10)",
                            },
                            "end": {
                                "type": "integer",
                                "description": "Ending account number (39)",
                            }
                        },
                        "description": "Range of account numbers to search for short-term assets"
                    },
                    "include_details": {
                        "type": "boolean",
                        "description": "Whether to include additional account details in the response",
                    }
                },
                "required": ["account_range"]
            }
        }
    }
]

# 2. Implement hàm thực tế để xử lý query
def get_short_term_assets(account_range: Dict[str, int], include_details: bool = False) -> str:
    try:
        # Tạo câu query SQL
        sql_query = """
        SELECT accno, accname
        FROM "ACCNO"
        WHERE CAST(accno AS INTEGER) BETWEEN %s AND %s
        ORDER BY accno;
        """
        
        # Thực thi query với sql_database đã được định nghĩa trước đó
        result = sql_database.run(sql_query, (account_range["start"], account_range["end"]))
        
        return result
    except Exception as e:
        return f"Error executing query: {str(e)}"

# 3. Hàm xử lý function calling
def process_function_call(function_call: Dict[str, Any]) -> str:
    function_name = function_call.get("name")
    function_args = json.loads(function_call.get("arguments", "{}"))
    
    if function_name == "get_short_term_assets":
        return get_short_term_assets(
            account_range=function_args.get("account_range"),
            include_details=function_args.get("include_details", False)
        )
    return "Function not found"

# 4. Sử dụng với OpenAI API
def get_short_term_assets_info(user_query: str) -> str:
    messages = [
        {"role": "system", "content": "You are a helpful assistant that helps users get information about short-term assets from the accounting system."},
        {"role": "user", "content": user_query}
    ]
    
    response = openai.chat.completions.create(
        model="gpt-4o-mini",  # hoặc model phù hợp khác
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    # Xử lý response
    tool_calls = response.choices[0].message.tool_calls
    if tool_calls:
        # Xử lý function call
        function_call = tool_calls[0].function
        result = process_function_call(function_call)
        return result
    
    return response.choices[0].message.content

# 5. Sử dụng function
try:
    # Ví dụ sử dụng
    query = "Tính tổng tài sản ngắn hạn của BID năm 2024 quý 2"
    result = get_short_term_assets_info(query)
    print("Kết quả:")
    print(result)
except Exception as e:
    print(f"Error: {str(e)}")

Error: module 'openai' has no attribute 'chat'


In [98]:
import json
from langchain import OpenAI
client = OpenAI()

tool = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
}

call_id = 'call_qwertyuiop'

exemplar_in = "Is it raining in Tokyo today?"
exemplar_out = {
    "location":"Tokyo",
    "unit":"celsius"
}

messages = [
    {'role': 'user', 'content': exemplar_in},
    {
        'role': 'assistant',
        'tool_calls': [{
            'id': call_id,
            'function': {
                'name': tool["function"]["name"],
                'arguments': json.dumps(exemplar_out)
            },
            'type': 'function'
        }],
    },
    {
        "role": "tool",
        "tool_call_id": call_id,
        "name": tool["function"]["name"],
        "content": json.dumps(exemplar_out),
    },
    {'role': 'user', 'content': "What's the current weather in Rome, Italy."},
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=messages,
    tools=[tool],
    # Force calling the function as a tool:
    tool_choice={
        "type": "function",
        "function": {
            "name": tool["function"]["name"]
        }
    },
)
response_message = response.choices[0].message
response_message.model_dump()

C:\Users\nn200\AppData\Local\Temp\ipykernel_24712\3269429102.py:3: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  client = OpenAI()


AttributeError: 'OpenAI' object has no attribute 'chat'

In [95]:
!pip install openai==0.28

  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
Using cached openai-0.28.0-py3-none-any.whl (76 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.52.0
    Uninstalling openai-1.52.0:
      Successfully uninstalled openai-1.52.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.2.3 requires openai<2.0.0,>=1.52.0, but you have openai 0.28.0 which is incompatible.


In [87]:
client = OpenAI()  # This will use your OPENAI_API_KEY environment variable

def get_short_term_assets_info(user_query: str) -> str:
    messages = [
        {"role": "system", "content": "You are a helpful assistant that helps users get information about short-term assets from the accounting system."},
        {"role": "user", "content": user_query}
    ]
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    # Xử lý response
    tool_calls = response.choices[0].message.tool_calls
    if tool_calls:
        # Xử lý function call
        function_call = tool_calls[0].function
        result = process_function_call(function_call)
        return result
    
    return response.choices[0].message.content

NameError: name 'OpenAI' is not defined